# Use Cases for VIGOR

This notebook is to demonstrate a use case for VIGOR

In [1]:
import csv
from neo4j import GraphDatabase
from neo4j.graph import Node, Relationship
from vigor import Graph, VIGOR, predicates, Visualization, Predicate

## Recommendation Use Case

Uses a sample database provided by Neo4j

In [2]:
vigor = VIGOR()

visualization_preds = {}
for vis, score, stat, min, max in predicates:
    if vis not in visualization_preds:
        visualization_preds[vis] = Visualization(vis)
    visualization_preds[vis].add_predicate(Predicate(stat, min, max, score))

for vis in visualization_preds.values():
    vigor.add_visualization(vis)

In [3]:
queries = {
    "recommendations": [
      {
        "task": "Topology-Based",
        "question": "Which director directed the most amount of movies?",
        "query": "MATCH (d:Director)-[r:DIRECTED]-(m:Movie) RETURN * LIMIT 500"
      },
      {
        "task": "Attribute-Based",
        "question": "Which of the movies has the heighest average rating by a user?",
        "query": "MATCH (u:User)-[r:RATED]-(m:Movie) RETURN * LIMIT 100"
      },
      {
        "task": "Browsing",
        "question": "Can you find all actors that also directed the movie they played in?",
        "query": "MATCH (a:Actor)-[r:ACTED_IN]->(m:Movie)-[d:DIRECTED]-(e:Actor) RETURN * LIMIT 1000"
      },
       {
        "task": "Overview",
        "question": "Can you estimate the size of the entire network?",
        "query": "MATCH (n)-[r]->(m) RETURN * LIMIT 1000"
      }
    ],
}

In [4]:
NEO4J_CONNECTION_URI="neo4j+s://demo.neo4jlabs.com:7687"
csv_filename = "../data/use-cases.csv"

In [5]:
header_written = False

with open(csv_filename, mode='a', newline='') as file:
    writer = csv.writer(file)

    for database in queries:
        with GraphDatabase.driver(NEO4J_CONNECTION_URI, auth=(database, database)) as driver:
            for query in queries[database]:
                G = Graph()

                # Execute the query and retrieve records
                records, summary, keys = driver.execute_query(query['query'], database_=database)

                for record in records:
                    for element in record:
                        if isinstance(element, Node):
                            labels = list(element.labels) if isinstance(element.labels, frozenset) else element.labels
                            node_attributes = {
                                key: (val[:10] if isinstance(val, str) and len(val) > 10 else val)
                                for key, val in element.items()
                                if "embedding" not in key.lower()
                            }
                            node_id = element.element_id
                            G.add_node(node_id, label=labels, **node_attributes)

                        elif isinstance(element, Relationship):
                            edge_attributes = {
                                key: (val[:10] if isinstance(val, str) and len(val) > 10 else val)
                                for key, val in element.items()
                                if "embedding" not in key.lower()
                            }
                            edge_type = element.type
                            G.add_edge(element.start_node.element_id, element.end_node.element_id, type=edge_type, **edge_attributes)

                # Calculate graph statistics and recommendations
                graph_stats = G.get_statistics()
                print(graph_stats)

                # Unpack recommendations and scores
                recommendations_with_scores = vigor.recommend(graph_stats, 8)
                recommendations = [rec[0].value for rec in recommendations_with_scores]
                scores = [rec[1] for rec in recommendations_with_scores]

                # Add recommendations and scores to graph stats
                graph_stats.update({
                    'database_name': database,
                    'task': query['task'],
                    'query': query['query'],
                    'question': query['question'],
                    "rec_1": recommendations[0],
                    "rec_2": recommendations[1],
                    "rec_last": recommendations[-1],
                    "rec_second_to_last": recommendations[-2],
                    "score_1": scores[0],
                    "score_2": scores[1],
                    "score_last": scores[-1],
                    "score_second_to_last": scores[-2]
                })

                # Write header if it hasn't been written
                if not header_written:
                    writer.writerow(graph_stats.keys())
                    header_written = True

                writer.writerow(graph_stats.values())

{'graph_type': 3, 'is_directed_int': 0, 'has_spatial_attributes': 0, 'has_temporal_attributes': 1, 'is_planar': 1, 'is_bipartite': 1, 'n_components': 114, 'avg_betweenness_centrality': 2.8341959255350203e-05, 'avg_closeness_centrality': 0.010367396422318965, 'avg_eigenvector_centrality': 0.008691461397179927, 'avg_degree': 1.6420361247947455, 'std_degree': 2.9272887680561324, 'clustering_coefficient': 0.0, 'transitivity': 0, 'modularity': 0.9711780000000003, 'communities': 115, 'avg_shortest_path_length': -1, 'radius': -1, 'diameter': -1, 'assortativity': -0.3250122093356985, 'vertex_connectivity': 0, 'eccentricity_avg': -1, 's_metric': -1, 'sigma': -1, 'n_nodes': 609, 'node_types': 2, 'node_attributes': 14.683087027914613, 'number_of_isolates': 0, 'density': 0.0027007173105176737, 'edge_types': 1, 'edge_attributes': 1.402, 'n_parallel_edges': 0, 'n_self_loops': 0}
{'graph_type': 3, 'is_directed_int': 0, 'has_spatial_attributes': 0, 'has_temporal_attributes': 1, 'is_planar': 1, 'is_bip

KeyboardInterrupt: 